# Derivation of `RWC_alam_clam()`
*Last updated on 2022-05-16 by Arthur Ryman*

# Introduction

I ran into a `solveset()` bug.

Here's the Maple behaviour:

```
RWC_alam(14, -3, 2);
                   [4.435803810, 15.83330432]
RWC_alam(15, -3, 2);
                   [4.572908819, 16.75518890]
```

Recreate this in Python.

In [1]:
from acmpy.globals import ACM_set_defaults
ACM_set_defaults(0)

In [2]:
from acmpy.hamiltonian_data import RWC_alam

RWC_alam(14, -3, 2)

(4.435803809483709, 15.833304312584458)

In [3]:
try:
    RWC_alam(15, -3, 2)
except ValueError as ve:
    print(ve)

max() arg is an empty sequence


As of 2022-05-15, the preceeding call raises an error:

```
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Input In [7], in <module>
----> 1 RWC_alam(15, -3, 2)

File ~/Documents/repositories/agryman/acmpy/src/acmpy/hamiltonian_data.py:176, in RWC_alam(B, c1, c2, v)
    170     def RWC2(aa: Expr, mu: Expr) -> Expr:
    171         return (c1 / c2) ** 2 * (-vshft * aa ** 5 / mu ** 2
    172                                  + aa ** 3 * B ** 2 * c1  + aa **2 * B ** 2 * c2 * (mu + 3)) \
    173                + aa ** 3 * (2 * mu + vshft) \
    174                - B ** 2 * mu * (mu + 2) * (aa * c1 + c2 * (mu + 4))
--> 176     aa0 = max(float(aa) for aa in solveset(RWC2(A, muf(A)), A, domain=S.Reals))
    177     return math.sqrt(aa0), float(1 + muf(S(aa0)) / 2)
    179 else:

ValueError: max() arg is an empty sequence
```

The error comes from `solveset()` returning an empty set of solutions.
However, I know that there is at least one solution.
There most probable explanation is that there is a bug in `solveset()`.

I'd like to report this bug to the SymPy project.
I therefore need to create a simple test case the reproduces the problem.
I tried to derive the expressions used in `RWC_alam()` but got something very 
complicated.

I'll first try to derive the formulas used in the simpler case of `RWC_alam_clam()`.

## `RWC_alam_clam()`

The Maple and Python code follows:

```
# # The following procedure RWC_alam_clam is another alternative that
# # returns values of the ACM parameters (anorm,lambda), which are
# # obtained from the minimal value of the expectation value of RWC_expt,
# # given above, with lambda assumed to take the constant value of 2.5
#
# RWC_alam_clam:=proc(B::constant,c1::constant,c2::constant,$)
#   local RWC1,RWC2,muf,aa0,A;
#
#     RWC1:=(aa) -> aa^3 - B^2*c1*aa - 7*B^2*c2:
#     aa0:=max(fsolve(RWC1(A)=0,A)):
#     return [sqrt(aa0),2.5]:
#
# end:
def RWC_alam_clam(B: IntFloatExpr, c1: IntFloatExpr, c2: IntFloatExpr
                  ) -> tuple[float, float]:
    B = S(B)
    c1 = S(c1)
    c2 = S(c2)

    def RWC1(aa: Expr) -> Expr:
        return aa ** 3 - B ** 2 * c1 * aa - 7 * B ** 2 * c2

    A: Symbol = symbols('A', real=True)
    aa0: float = max(float(aa) for aa in solveset(RWC1(A), A, domain=S.Reals))

    return math.sqrt(aa0), 2.5
```

Here's the Maple behaviour.

```
RWC_alam_clam(14, -3, 2);
                       [2.123816133, 2.5]
RWC_alam_clam(15, -3, 2);
                       [2.128153304, 2.5]
```

In [4]:
from acmpy.hamiltonian_data import RWC_alam_clam

RWC_alam_clam(14, -3, 2)

(2.123816132523058, 2.5)

In [5]:
RWC_alam_clam(15, -3, 2)

(2.1281533037939857, 2.5)

## Mathematical Definitions

Here are the mathematical formulae encoded in SymPy.

In [6]:
from sympy import *

B = Symbol('B', real=True, positive=True)
c1 = Symbol('c1', real=True)
c2 = Symbol('c2', real=True)
lambda0 = Symbol('lambda0', real=True, positive=True)
a = Symbol('a', real=True, positive=True)
A = Symbol('A', real=True, positive=True)
E = Symbol('E', real=True)

In [7]:
A_a = a ** 2
A_a

a**2

In [8]:
E1_A_lambda0 = A / (2 * B) * (1 + 9 / (4 * (lambda0 - 1)))
E1_A_lambda0

A*(1 + 9/(4*lambda0 - 4))/(2*B)

In [9]:
E1_a_lambda0 = E1_A_lambda0.subs(A, A_a)
E1_a_lambda0

a**2*(1 + 9/(4*lambda0 - 4))/(2*B)

In [10]:
E2_A_lambda0 =  B / (2 * A) * c1 * lambda0
E2_A_lambda0

B*c1*lambda0/(2*A)

In [11]:
E3_A_lambda_0 = B / (2 * A ** 2) * c2 * lambda0 * (lambda0 + 1)
E3_A_lambda_0

B*c2*lambda0*(lambda0 + 1)/(2*A**2)

In [12]:
E_A_lambda0 = E1_A_lambda0 + E2_A_lambda0 + E3_A_lambda_0
E_A_lambda0

A*(1 + 9/(4*lambda0 - 4))/(2*B) + B*c1*lambda0/(2*A) + B*c2*lambda0*(lambda0 + 1)/(2*A**2)

Now substitute $\lambda_0 = 5/2$ in the energy.

In [13]:
E_A = E_A_lambda0.subs(lambda0, Rational(5, 2))
E_A

5*A/(4*B) + 5*B*c1/(4*A) + 35*B*c2/(8*A**2)

The minimum energy occurs when the derivative vanishes.

In [14]:
Z = E_A.diff(A)
Z

5/(4*B) - 5*B*c1/(4*A**2) - 35*B*c2/(4*A**3)

Multiply through by $4/5$ to simplify the expression.

In [15]:
Z1 = Z * Rational(4, 5)
Z1

1/B - B*c1/A**2 - 7*B*c2/A**3

Multiply through by $B A^3$ to clear the denominators.

In [16]:
Z2 = simplify(Z1 * B * A ** 3)
Z2

A**3 - A*B**2*c1 - 7*B**2*c2

The `RWC_alam_clam()` code finds zeroes of the following expression:

```
RWC1:=(aa) -> aa^3 - B^2*c1*aa - 7*B^2*c2
```

## Conclusion

The SymPy derivation agrees with the Maple code for `RWC_alam_clam()`.